# Financial Analysis Notebook

This notebook performs profitability and growth analysis on financial data from multiple CSV files.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import ExcelWriter
from IPython.display import display, HTML

# Set style for seaborn and matplotlib
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

In [2]:
# List of file paths to compare (add paths as needed)
file_paths = [
    "tcs.csv",
    "zeel.csv",
    "tcs1.csv",
    "zeel1.csv"
]

# Date format for conversion
date_format = "%b-%y"

# Data containers
all_profit_data = {}
all_growth_data = {}
comparative_metrics = {}

In [3]:
# Analysis functions
def profitability_analysis(df):
    df['Gross Margin'] = (df['Total Revenue/Income'] - df['Total Operating Expense']) / df['Total Revenue/Income'] * 100
    df['Operating Margin'] = df['Operating Income/Profit'] / df['Total Revenue/Income'] * 100
    df['Net Profit Margin'] = df['Net Income'] / df['Total Revenue/Income'] * 100
    return df[['Date', 'Gross Margin', 'Operating Margin', 'Net Profit Margin']]

def growth_analysis(df):
    df['Revenue Growth'] = df['Total Revenue/Income'].pct_change() * 100
    df['Net Income Growth'] = df['Net Income'].pct_change() * 100
    return df[['Date', 'Revenue Growth', 'Net Income Growth']]

In [4]:
# Process each file and perform analysis
for file_path in file_paths:
    data = pd.read_csv(file_path)

    # Convert Date column and sort
    data['Date'] = pd.to_datetime(data['Date'], format=date_format, errors='coerce')
    data.dropna(subset=['Date'], inplace=True)
    data.sort_values('Date', inplace=True)

    # Perform analyses
    profit_data = profitability_analysis(data)
    growth_data = growth_analysis(data)
    
    # Add file identifier
    file_id = file_path.split('/')[-1].replace('.csv', '')
    
    # Store data in dictionaries with file-based keys
    all_profit_data[f"{file_id} Combined Profitability"] = profit_data
    all_growth_data[f"{file_id} Combined Growth"] = growth_data
    
    # Store comparative metrics
    comparative_metrics[file_id] = {
        "Avg Gross Margin": profit_data['Gross Margin'].mean(),
        "Avg Operating Margin": profit_data['Operating Margin'].mean(),
        "Avg Net Profit Margin": profit_data['Net Profit Margin'].mean(),
        "Avg Revenue Growth": growth_data['Revenue Growth'].mean(),
        "Avg Net Income Growth": growth_data['Net Income Growth'].mean()
    }

In [5]:
# Convert comparative metrics to DataFrame for easier export
comparative_df = pd.DataFrame(comparative_metrics).T
comparative_df.index.name = "File"
comparative_df.reset_index(inplace=True)

# Save results to Excel
output_file = "financial_comparison_analysis.xlsx"
with ExcelWriter(output_file) as writer:
    for sheet_name, df in all_profit_data.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)
    for sheet_name, df in all_growth_data.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)
    comparative_df.to_excel(writer, sheet_name="Comparative Metrics", index=False)

In [6]:
# Plot comparison graphs
plt.figure(figsize=(12, 8))
comparative_df.plot(x="File", kind="bar", stacked=False)
plt.title("Comparison of Key Financial Metrics Across Files")
plt.ylabel("Metric Averages")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("financial_metric_comparison.png")
plt.close()

In [7]:
# Output completion message
print(f"Analysis complete. Results saved to {output_file}.")

Analysis complete. Results saved to financial_comparison_analysis.xlsx.
